# Finite Elements Lab 1 Worksheet

##### Georges Limbert

###### nCATS, Faculty of Engineering and the Environment, University of Southampton

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

## Aims

The aim of this computer laboratory is to gain a practical insight into the fundamental steps required to solve a partial differential equation (PDE) over a simple geometric domain using a finite element approach. In doing so, you will learn:

* How to recast the PDE problem into a variational form 
* How to define simple domains
* How to deal with Dirichlet boundary conditions
* How to numerically solve the variational problem.
* How to compute derived quantities from the solution.
* How to visualise the mesh, the solution and derived quantities

The symbolic‐numeric environment chosen for this laboratory is [FEniCS](http://fenicsproject.org/) which is a powerful and user‐friendly tool for solving PDEs. *However*, FEniCS is not compatible with Anaconda on a Mac it would seem; to use it, you must temporarily comment out the line setting up Anaconda in your `.bash_profile`.

## Basic problem: the Poisson equation


The Poisson equation is encountered in a wide range of domains in physics from heat conduction, electrostatics, diffusion and elasticity. The Poisson problem is defined as follows:

$$
\begin{align}
  - {\nabla ^2}u(x) &= f(x) & x & \text{ in } \Omega \\
  u(x) &= {u_0}(x) & x &\text{ on } \partial \Omega
\end{align}
$$

Here, $u(x)$ is the unknown function, $f$ and ${u_0}$ are prescribed functions, ${\nabla^2} = \Delta $ is the Laplace operator, $\Omega $ is the spatial domain and $\partial \Omega $ is its boundary.

A *time-independent* or *stationary* PDE like the Poisson equation combined with boundary conditions such as those above constitute a **boundary-value problem**.

## FEnics workflow

Solving a physical problem with FEnics involves the following workflow:

1. Identify the PDE and its boundary conditions
2. Recast the PDE problem into a variational problem
3. Write a Python programme encoding the :

   1. variational problem
   2. definition of input data: e.g. $f$ and ${u_0}$
   3. finite element mesh for the spatial domain $\Omega $:
   
4. Add statements to the Python programme for:

   1. solving the variational problem
   2. computing derived quantities such as the gradient of the unknown function
   3. visualising the results

## Variational formulation

The core of the recipe for turning a PDE into a variational problem is to multiply the PDE by a function $\upsilon $, integrate the resulting equation over $\Omega $, and perform integration by parts of terms with second-order derivatives.

The function $\upsilon$ which multiplies the PDE is in the mathematical finite element literature called a **test function**. The unknown function $u$ to be approximated is referred to as a **trial function**.

The terms test and trial function are used in FEniCS programs too.

The test function $\upsilon$ is required to vanish on the parts of the boundary where $u$ is known

**Suitable function spaces** must be specified for the test and trial functions. For standard PDEs arising in physics and mechanics such spaces are well known.

$$
\begin{equation}
  - \int_\Omega  \nabla^2 u(x)\upsilon \, dx  = \int_\Omega  f(x)\upsilon \, dx
\end{equation}
$$

### Question 1


Apply integration by parts to the PDE $ - {\nabla ^2}u(x) = f(x)$ with the
divergence theorem (relation between the flow (flux) of a vector field though a
surface and the behavior inside the surface) to recast it into its variational form.
You will also take into account the conditions about the test function $\upsilon
$ on the boundary where $u$ is prescribed.


You have now derived the variational formulation of the Poisson problem. This
variational form is what is commonly called a **weak form** of the
boundary-value problem ("weak" because of the reduced continuity requirements
placed on $u$). It contains the basic equation $ - {\nabla ^2}u(x) = f(x)$ and
the condition $u = {u_0} \text{ on }\partial \Omega $.

This continuous variational problem must be discretised to be solved using the
finite element method. The continuous version of the unknown function $u$ will be
denoted by ${u_e}$ (the index $e$ stands for "exact") to make the
distinction with the discretised solution of the problem that is typically
denoted by ${u_h}$.

**NB:** In the context of FEnics, $u$ will be meant to represent the
discretised approximate solution of the weak form.


For a linear weak form like the one just established it proves convenient to
introduce the following notation:

$$
\begin{equation}
a(u,\upsilon ) = \int_{\Omega}  {\nabla u \cdot \nabla \upsilon \, dx}
\end{equation}
$$

$$
\begin{equation}
L(\upsilon ) = \int_{\Omega}  {f\upsilon \, dx}
\end{equation}
$$

Where $a$ and $L$ are respectively a **bilinear** and **linear operator**.


The weak form of the Poisson problem (or any linear weak form) can therefore be
expressed as:

$$
\begin{equation}
a(u,\upsilon ) = L(\upsilon )
\end{equation}
$$

For every linear problem to be solved:

* all the terms containing the unknown $u$  must be collected in $a(u,\upsilon )$
* all terms with only known functions must be collected in $L(\upsilon )$

The explicit formulas for $a$ and $L$ are then coded in the programme.

Solving the discretised weak form consists in finding:

$$
\begin{equation}
  u \in V \text{ such that } a(u, \upsilon) = L(\upsilon) \quad \forall \upsilon \in V
\end{equation}
$$

Note that

$$
\begin{equation}
  V = \{ v \in H^1(\Omega): v = u_0 \text{ on } \partial \Omega \}
\end{equation}
$$

is the space of **trial** functions, and

$$
\begin{equation}
  \hat{V} = \{ v \in H^1(\Omega): v = 0 \text{ on } \partial \Omega \}
\end{equation}
$$

is the space of **test** functions.

Specifying $V$ and $\hat{V}$ consists in choosing the mesh and the type of interpolation functions.


## Practical implementation on a concrete example


Here we are going to particularise the Poisson problem (reproduced here for
convenience):

$$
\begin{align}
  - {\nabla ^2}u(x) &= f(x) & x & \text{ in } \Omega \\
  u(x) &= {u_0}(x) & x &\text{ on } \partial \Omega
\end{align}
$$


This decision means that $\{ {u_0},f,\Omega \} $ needs to be defined.

We choose a simple 2D domain: the unit square $\Omega  = [0,1] \times [0,1]$ .

In order to compare the finite element solution to the exact solution we also
*a priori* select a particular form for $u$. We choose:

$$
\begin{equation}
{u_e}(x,y) = 1 + {x^2} + 2{y^2}
\end{equation}
$$


If we inject equation the exact solution into Poisson problem we find that ${u_e}$  is solution if:

$$
\begin{equation}
f(x,y) =  - 6; \qquad {u_0}(x,y) = {u_e}(x,y) = 1 + {x^2} + 2{y^2}
\end{equation}
$$

### Question 2

Using FEnics write an algorithm to solve the boundary-value problem described above.

1.	Create and visualise the mesh using first-order (linear) triangular elements. You will use (6,4) as arguments for the mesh function which means that the computational domain will be first divide into 24 rectangles which will be further split into triangles
2.	Set up the boundary conditions and visualise them
3.	Solve the problem
4.	Plot the approximate solution over the domain using colour plots.
5.	Plot the error between the exact and approximate solution over the domain using colour plots
6.	Calculate the maximum error for the whole domain
7.	Calculate and plot the gradient of $u$


## A more concrete example: a pressurised elastic membrane


Here, we study the deflection $D(x,y)$ of an elastic circular membrane with
radius $R$, subject to a localized perpendicular pressure force, modelled as a
Gaussian function. The PDE representing this physical process is:

$$
\begin{equation}
- T{\nabla ^2}D = p(x,y) \text{ in } \Omega  = \{ (x,y)|\,{x^2} + {y^2} \le R\}
\end{equation}
$$

where $p$ is the external pressure defined as:

$$
\begin{equation}
p(x,y) = \frac{A}{{2\pi \sigma }}\exp \left[ { - \frac{1}{2}{{\left( {\frac{{x -
{x_0}}}{\sigma }} \right)}^2} - \frac{1}{2}{{\left( {\frac{{y - {y_0}}}{\sigma }}
\right)}^2}} \right]
\end{equation}
$$

$T$ the constant tension in the membrane, $A$ the amplitude of the pressure,
$({x_0},{y_0})$ the localisation of the Gaussian pressure function and $\sigma $
its width.

You will use the following values:

$$
\begin{align}
T &= 10 \\ A &= 1 \\ R &= 0.3 \\ \theta &= 0.2 \\ {x_0} &= 0.6R\cos (\theta ) \\ {y_0} &= 0.6R\sin
(\theta ) \\ \sigma  &= 0.025 
\end{align}
$$

### Question 3

1.	Determine an analytical solution for the above equation equation 
2.	Using the previously written Python programme, make appropriate modifications to solve the membrane problem.
3.	Solve the problem
4.	Calculate the maximum deflection
5.	Plot the approximate solution over the domain using colour plots.
6.	Plot the error between the exact and approximate solution over the domain using colour plots
7.	Calculate the maximum error for the whole domain
8.	Calculate and plot the gradient of $u$.